In [1]:
import paddle as paddle
import paddle.fluid as fluid
import numpy as np
import matplotlib.pyplot as plt
import sys
import cv2
import os
import random
import math
import PIL.Image as Image
from paddle.fluid.initializer import MSRA
from paddle.fluid.param_attr import ParamAttr
from visualdl import LogWriter

In [2]:
data_shape = [128,128]


class AlexNet():
    def __init__(self):
        pass

    def net(self, input, class_dim=1000):
        stdv = 1.0 / math.sqrt(input.shape[1] * 5 * 5)
        conv1 = fluid.layers.conv2d(
            input=input,
            num_filters=64,
            filter_size=5,
            stride=1,
            padding=2,
            groups=1,
            act='relu',
            bias_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)),
            param_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)))
        pool1 = fluid.layers.pool2d(
            input=conv1,
            pool_size=3,
            pool_stride=2,
            pool_padding=0,
            pool_type='max')

        stdv = 1.0 / math.sqrt(pool1.shape[1] * 5 * 5)
        conv2 = fluid.layers.conv2d(
            input=pool1,
            num_filters=192,
            filter_size=5,
            stride=1,
            padding=2,
            groups=1,
            act='relu',
            bias_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)),
            param_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)))
        pool2 = fluid.layers.pool2d(
            input=conv2,
            pool_size=3,
            pool_stride=2,
            pool_padding=0,
            pool_type='max')

        stdv = 1.0 / math.sqrt(pool2.shape[1] * 3 * 3)
        conv3 = fluid.layers.conv2d(
            input=pool2,
            num_filters=384,
            filter_size=3,
            stride=1,
            padding=1,
            groups=1,
            act='relu',
            bias_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)),
            param_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)))

        stdv = 1.0 / math.sqrt(conv3.shape[1] * 3 * 3)
        conv4 = fluid.layers.conv2d(
            input=conv3,
            num_filters=256,
            filter_size=3,
            stride=1,
            padding=1,
            groups=1,
            act='relu',
            bias_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)),
            param_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)))

        stdv = 1.0 / math.sqrt(conv4.shape[1] * 3 * 3)
        conv5 = fluid.layers.conv2d(
            input=conv4,
            num_filters=256,
            filter_size=3,
            stride=2,
            padding=1,
            groups=1,
            act='relu',
            bias_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)),
            param_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)))
        pool5 = fluid.layers.pool2d(
            input=conv5,
            pool_size=3,
            pool_stride=2,
            pool_padding=0,
            pool_type='max')
        print(pool5)
        drop6 = fluid.layers.dropout(x=pool5, dropout_prob=0.5)

        stdv = 1.0 / math.sqrt(drop6.shape[1] * drop6.shape[2] *
                               drop6.shape[3] * 1.0)
        fc6 = fluid.layers.fc(
            input=drop6,
            size=256,
            act='relu',
            bias_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)),
            param_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)))

        drop7 = fluid.layers.dropout(x=fc6, dropout_prob=0.5)

        stdv = 1.0 / math.sqrt(drop7.shape[1] * 1.0)
        fc7 = fluid.layers.fc(
            input=drop7,
            size=128,
            act='relu',
            bias_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)),
            param_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)))

        stdv = 1.0 / math.sqrt(fc7.shape[1] * 1.0)
        out = fluid.layers.fc(
            input=fc7,
            size=class_dim,
            bias_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)),
            param_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)),act='softmax')
        return out

In [3]:
yuzhi = 1
def addGaussianNoise(image,percetage): 
    G_Noiseimg = image 
    G_NoiseNum=int(percetage*image.shape[0]*image.shape[1]) 
    for i in range(G_NoiseNum): 
        temp_x = np.random.randint(0,image.shape[0]) 
        temp_y = np.random.randint(0,image.shape[0])
        G_Noiseimg[temp_x][temp_y] = 255
    return G_Noiseimg
def for_iterater_reader(t_list):
    def reader():
        for i in range(0,4000):
            for label in range(0,21):
              #  try:
                    #img = cv2.resize(img,(data_shape[0],data_shape[1]))
                    #img = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
                    #img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                    if label == 10:#没有10这个数
                        continue
                    tmp_ran = train_list[label]
                    ran_int = random.randint(0,len(tmp_ran)-1)#取随机的一个数
                    img = np.array(tmp_ran[ran_int]).reshape(28,28)
                    img = Image.fromarray(img).resize((data_shape[0],data_shape[0]))
                    img = np.array(img)
                    _, img = cv2.threshold(img, 10, 255,cv2.THRESH_BINARY)
                    if random.randint(0,1)==1:
                        img = addGaussianNoise(img,random.randint(0,3)/100)
                    r_x = random.randint(-5,5)
                    r_y = random.randint(-5,5)
                    r_rota = random.randint(-5,5)
                    r_yh = random.randint(1,3)
                    img = Image.fromarray(img)
                    img = img.rotate(r_rota)
                    tmpimg = Image.new('L',(data_shape[0],data_shape[0]))
                    tmpimg.paste(img,(r_x,r_y))
                    element1 = cv2.getStructuringElement(cv2.MORPH_RECT, (r_yh, r_yh))
                    tmpimg = cv2.dilate(np.array(tmpimg), element1, iterations = 1)
                    #img = cv2.blur(img,(20,20))#羽化
                    #ret, binary = cv2.threshold(img, 50, 255,cv2.THRESH_BINARY)
                    #element1 = cv2.getStructuringElement(cv2.MORPH_RECT, (10,10))#拓展
                    #img = cv2.erode(binary, element1, iterations = 1)
                    img = np.array(tmpimg)
                    ######################################################
                    img = img.reshape(1,data_shape[0],data_shape[1]).astype('float32')
                    img = (img)/255 
                    yield label,img
            #    except Exception:
             #       print(label)
    return reader

def for_test_reader(t_list):
    def reader():
        for class_index in t_list:
            image = np.array(class_index[1]).reshape(28,28)
            image = Image.fromarray(image).resize((data_shape[0],data_shape[0]))
            image = np.array(image).reshape(1,data_shape[0],data_shape[0]).astype('float32')
            image = (image) / 255.0
            yield class_index[0],image
    return reader

In [4]:
#1.设置地方
place = fluid.CUDAPlace(0)
#2.设置数据和设置标签
label = fluid.layers.data(name = 'label',shape=[1],dtype='int64')
image = fluid.layers.data(name = 'image',shape=[1,data_shape[0],data_shape[1]],dtype='float32')
#3.设置网络和Feeder
feeder = fluid.DataFeeder(place = place , feed_list = [label,image])
net = AlexNet().net(image,21)
#4.设置损失函数和正确率
cost = fluid.layers.cross_entropy(input = net , label = label)
avg_cost = fluid.layers.mean(cost)
acc = fluid.layers.accuracy(input = net , label =label)
#.定义测试程序
test_program = fluid.default_main_program().clone(for_test=True)
#5.设置优化
optimizer = fluid.optimizer.Adam(learning_rate=0.001)
optimizer.minimize(avg_cost)
#6.定义Executor
exe = fluid.Executor(place = place)
exe.run(fluid.default_startup_program())

name: "pool2d_2.tmp_0"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: -1
      dims: 256
      dims: 7
      dims: 7
    }
    lod_level: 0
  }
}
persistable: false



[]

In [5]:
import shutil
import os
def save_inference(pass_id):
     # 保存预测模型
    save_path = 'models/infer_model/'+str(pass_id)+"/"
    # 删除旧的模型文件
    shutil.rmtree(save_path, ignore_errors=True)
    # 创建保持模型文件目录
    os.makedirs(save_path)
    # 保存预测模型
    fluid.io.save_inference_model(save_path, feeded_var_names=[image.name], target_vars=[net], executor=exe)

def save_model(pass_id):
    # 保存参数模型
    save_path = 'models/params_model/'+str(pass_id)+"/"
    # 删除旧的模型文件
    shutil.rmtree(save_path, ignore_errors=True)
    # 创建保持模型文件目录
    os.makedirs(save_path)
    # 保存参数模型
    fluid.io.save_params(executor=exe, dirname=save_path)

def load_model(pass_id):
    # 加载之前训练过的参数模型
    save_path = 'models/params_model/'+str(pass_id)
    if os.path.exists(save_path):
        print('使用参数模型作为预训练模型')
        fluid.io.load_params(executor=exe, dirname=save_path)

In [6]:
def produce_path_label(path):
    train_list = [[] * 1 for i in range(0,21)]
    train_data = open(path).readlines()
    for i in train_data:
        items = i.split(',')
        class_index = int(items[0])
        img = np.array(items[1:]).astype('uint8')
        train_list[class_index].append([img])
    return train_list

def produce_test(path):
    train_list = []
    train_data = open(path).readlines()
    for i in train_data:
        items = i.split(',')
        class_index = int(items[0])
        img = np.array(items[1:]).astype('uint8')
        train_list.append([class_index,img])
    return train_list
train_list = produce_path_label('train_list.txt')
test_list = produce_path_label('test_list.txt')
true_test_list = produce_test('true_test_list.txt')

In [7]:
#8.数据分批
train_Reader = paddle.batch(reader=paddle.reader.shuffle(for_iterater_reader(train_list),buf_size = 128*128),batch_size=32)
test_Reader = paddle.batch(reader=paddle.reader.shuffle(for_iterater_reader(test_list),buf_size = 128*128),batch_size=32)
true_test_Reader = paddle.batch(reader=paddle.reader.shuffle(for_test_reader(true_test_list),buf_size = 128*128),batch_size=32)

In [8]:

print("加载完毕")

加载完毕


In [9]:
#9.训练
step = 0
best_model_precent = 0
error_cout = 20
save_model_name = 102

In [10]:
load_model(save_model_name)

In [11]:
 def train():
    for pass_id in range(20000):
        sum_cost=0
        sum_acc=0
        global best_model_precent
        for batch_id , data in enumerate(train_Reader()):
            train_cost, train_acc = exe.run(program = fluid.default_main_program(),
                                           feed = feeder.feed(data),
                                            fetch_list = [avg_cost,acc]
                                           )
            sum_cost = sum_cost + train_cost[0]
            sum_acc = sum_acc + train_acc[0]
            if batch_id % 50 == 0 and batch_id != 0:
                print('Pass：%d, Batch：%d, Cost：%f, Accuracy：%f' % (pass_id , batch_id , sum_cost/51, sum_acc/51))
                sum_cost = 0
                sum_acc = 0
        
        test_sum_cost=0
        test_sum_acc=0
        test_cout = 0
        for batch_id , data in enumerate(test_Reader()):
            train_cost, train_acc = exe.run(program = test_program,
                                           feed = feeder.feed(data),
                                            fetch_list = [avg_cost,acc]
                                           )
            test_sum_cost = test_sum_cost + train_cost[0]
            test_sum_acc = test_sum_acc + train_acc[0]
            test_cout+=1
        print("test - Pass:",pass_id,"Cost:",test_sum_cost/test_cout,"Acc:",test_sum_acc/test_cout)
        true_sum_cost=0
        true_sum_acc=0
        true_cout = 0
        for batch_id , data in enumerate(true_test_Reader()):
            train_cost, train_acc = exe.run(program = test_program,
                                           feed = feeder.feed(data),
                                            fetch_list = [avg_cost,acc]
                                           )
            true_sum_cost = true_sum_cost + train_cost[0]
            true_sum_acc = true_sum_acc + train_acc[0]
            true_cout+=1
        print("true test - Pass:",pass_id,"Cost:",true_sum_cost/true_cout,"Acc:",true_sum_acc/true_cout)
        total_model_precent = (test_sum_acc/test_cout)
        print("new BestPrecent:",best_model_precent,"thie Precent:",total_model_precent)
        if total_model_precent > best_model_precent:
            best_model_precent = total_model_precent
            save_model(save_model_name)
            print("Save...model...Best_precent:",best_model_precent)
        print()
train()

Pass：0, Batch：50, Cost：2.986723, Accuracy：0.080882
Pass：0, Batch：100, Cost：2.046915, Accuracy：0.329657
Pass：0, Batch：150, Cost：1.362880, Accuracy：0.541667
Pass：0, Batch：200, Cost：1.024309, Accuracy：0.656863
Pass：0, Batch：250, Cost：0.855256, Accuracy：0.704657
Pass：0, Batch：300, Cost：0.814348, Accuracy：0.712010
Pass：0, Batch：350, Cost：0.611622, Accuracy：0.784314
Pass：0, Batch：400, Cost：0.572955, Accuracy：0.804534
Pass：0, Batch：450, Cost：0.521891, Accuracy：0.816789
Pass：0, Batch：500, Cost：0.422741, Accuracy：0.843137
Pass：0, Batch：550, Cost：0.468103, Accuracy：0.848039
Pass：0, Batch：600, Cost：0.428219, Accuracy：0.846201
Pass：0, Batch：650, Cost：0.379398, Accuracy：0.861520
Pass：0, Batch：700, Cost：0.391719, Accuracy：0.860907
Pass：0, Batch：750, Cost：0.353188, Accuracy：0.869485
Pass：0, Batch：800, Cost：0.358046, Accuracy：0.870098
Pass：0, Batch：850, Cost：0.361862, Accuracy：0.871324
Pass：0, Batch：900, Cost：0.331138, Accuracy：0.882966
Pass：0, Batch：950, Cost：0.339027, Accuracy：0.882966
Pass：0, Batch

KeyboardInterrupt: 

In [ ]:
save_model(save_model_name)

In [ ]:
load_model(save_model_name)

In [12]:
save_inference(save_model_name)